In [1]:
# Mount gdrive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Change Directory

In [4]:
%cd "/content/drive/My Drive/UCM/tfm_code/notebooks"

/content/drive/My Drive/UCM/tfm_code/notebooks


In [5]:
# Import Libraries

In [6]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.6 MB/s eta 0:00:00


In [7]:
# Downloaded
import optuna
import pandas as pd
import pyarrow
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import  StratifiedKFold
import matplotlib.pyplot as plt

In [8]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances

In [9]:
from warnings import filterwarnings
filterwarnings('ignore')

In [10]:
# Codigo

In [11]:
df = pd.read_parquet(engine="pyarrow", path="/content/drive/My Drive/UCM/tfm_code/data/data_modelo_train_cleaned.parquet")

In [12]:
df = df[df['SEMANA'].isin([17,16,15,14])]
df.drop(columns=['CLIENTE_ID','PRODUCTO_ID','ANIO','SEMANA'], inplace=True)

In [13]:
important_columns = ['FRECUENCIA', 'RECENCY', 'DURATION_CLIENT', 'TAMANIO', 'CANAL_RSR',
       'MARCA_CIEL', 'MARCA_COCA-COLA', 'MARCA_DEL VALLE',
       'MARCA_DELAWARE PUNCH', 'MARCA_FANTA', 'MARCA_FRESCA',
       'MARCA_FRUTSI', 'MARCA_FUZE', 'MARCA_MEZCLADA/MULTIPRODUC',
       'MARCA_MONSTER - PREDATOR', 'MARCA_MUNDET', 'MARCA_NADAS',
       'MARCA_OTROS', 'MARCA_POWERADE', 'MARCA_SANTA CLARA',
       'MARCA_SPRITE', 'MARCA_TOPO CHICO', 'MARCA_YOLI - SENZAO-AMEYAL',
       'SABOR_FRUTAS', 'SABOR_LIMA-LIMON', 'SABOR_MANZANA', 'SABOR_MORAS',
       'SABOR_NARANJA', 'SABOR_NATURAL', 'SABOR_OTROS', 'SABOR_UVA',
       'EMPAQUE_REF PET', 'EMPAQUE_TETRAPACK', 'EMPAQUE_VIDRIO',
       'TIPO_CONSUMO_PERSONALES (SS)', 'CLUSTER_1', 'CLUSTER_2',
       'CLUSTER_3', 'ROTATION_MEAN_CF', 'ROTATION_MEAN_DAYS', 'CF_LOG']

In [14]:
X = df.drop(columns=['PURCHASE'], axis=1)
X = X[important_columns]
y = df['PURCHASE']

In [15]:
np.random.seed(1234)

In [16]:
# Split your data into features (X) and target variable (y)
X_train, X, y_train, y = train_test_split(X, y, test_size=0.14, stratify=y,random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,random_state=42)

In [17]:
X_train['FRECUENCIA'].count()

211164

In [18]:
def objective(trial):
    params = {
        'penalty': trial.suggest_categorical('penalty',  ['l1','l2','none']),
        'C': trial.suggest_loguniform('C', 1e-4, 1e4),
        'solver': trial.suggest_categorical('solver', ['saga']),
        'max_iter': trial.suggest_int('max_iter', 50, 300),
        'random_state': 42
    }
    # Train and evaluate the model using the given hyperparameters
    clf = LogisticRegression(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    return f1

# Run hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters and the corresponding ROC AUC score
print('Best Average Precision Score:', study.best_value)
print('Best hyperparameters:', study.best_params)


[I 2023-03-17 13:53:57,892] A new study created in memory with name: no-name-b46e705f-628b-4f42-8c4e-1b1b597a41b5
[I 2023-03-17 13:54:18,606] Trial 0 finished with value: 0.5435170384784674 and parameters: {'penalty': 'none', 'C': 0.020969468499665914, 'solver': 'saga', 'max_iter': 90}. Best is trial 0 with value: 0.5435170384784674.
[I 2023-03-17 13:54:33,377] Trial 1 finished with value: 0.5435170384784674 and parameters: {'penalty': 'none', 'C': 0.021730907670955084, 'solver': 'saga', 'max_iter': 86}. Best is trial 0 with value: 0.5435170384784674.
[I 2023-03-17 13:54:48,358] Trial 2 finished with value: 0.5435170384784674 and parameters: {'penalty': 'none', 'C': 0.03703739830668169, 'solver': 'saga', 'max_iter': 298}. Best is trial 0 with value: 0.5435170384784674.
[I 2023-03-17 13:55:04,754] Trial 3 finished with value: 0.5435170384784674 and parameters: {'penalty': 'l1', 'C': 7213.128042916171, 'solver': 'saga', 'max_iter': 277}. Best is trial 0 with value: 0.5435170384784674.
[I

Best Average Precision Score: 0.5447652511616079
Best hyperparameters: {'penalty': 'l2', 'C': 0.10988725036679024, 'solver': 'saga', 'max_iter': 203}


In [19]:
plot_optimization_history(study)

In [20]:
plot_param_importances(study)